# Motor SQL

In [ ]:
import csvms.engine

In [ ]:
%%sql
CREATE TABLE lista_frutas (
    nm_fruta TEXT ,
    tp_fruta TEXT )


In [ ]:
%%sql
INSERT INTO lista_frutas VALUES ('banana','doce');
INSERT INTO lista_frutas VALUES ('limão','azedo');
INSERT INTO lista_frutas VALUES ('maçã','doce');

In [ ]:
%%sql
COMMIT;

In [ ]:
%%sql
DELETE FROM lista_frutas WHERE nm_fruta = 'limão'

In [ ]:
%%sql
UPDATE lista_frutas 
   SET tp_fruta = 'amargo'
 WHERE tp_fruta = 'doce';


In [ ]:
%%sql
ROLLBACK;

In [ ]:
%%sql
DROP TABLE lista_frutas;

Dica de biblioteca para ajudar no parse do SQL: <https://pypi.org/project/mo-sql-parsing>